# COVID-19 Genetic Risk Factor Analysis Tutorial

## Computational Network Analysis of Host Genetic Risk Variants

This tutorial is based on the research paper:
**"Computational network analysis of host genetic risk variants of severe COVID-19"**  
*Alsaedi, S.B., Mineta, K., Gao, X., Gojobori, T. (2023). Human Genomics 17, 17.*

### Overview
This notebook demonstrates a four-phase computational workflow for analyzing genetic risk variants associated with severe COVID-19 outcomes:

1. **Data Curation and Annotation**
2. **Functional Enrichment Analysis**
3. **Molecular Network Construction**
4. **Pathway Analysis and Disease Mapping**

### Learning Objectives
By the end of this tutorial, you will be able to:
- Load and preprocess genetic variant data
- Perform functional enrichment analysis
- Construct protein-protein interaction networks
- Analyze molecular pathways and disease associations
- Visualize genetic risk networks

## Setup and Installation

First, let's install the required packages for our analysis.

In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn plotly
!pip install networkx scipy scikit-learn
!pip install requests beautifulsoup4
!pip install bioservices gprofiler-official

# For Google Colab compatibility
import sys
if 'google.colab' in sys.modules:
    !pip install --upgrade plotly
    from google.colab import files

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import networkx as nx
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import requests
import json
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("All packages imported successfully!")

## Phase 1: Data Curation and Annotation

In this phase, we load and examine the curated dataset of genetic risk variants associated with severe COVID-19.

In [ ]:
# Load the COVID-19 genetic risk variants dataset
# In Google Colab, you can upload the CSV file or load from a URL

# Option 1: Load from uploaded file (uncomment if uploading)
# if 'google.colab' in sys.modules:
#     uploaded = files.upload()
#     df = pd.read_csv(list(uploaded.keys())[0])

# Option 2: Create sample data for demonstration
# This simulates the dataset structure from the paper
np.random.seed(42)

# COVID-19 related genes from literature
covid_genes = [
    "ACE2", "TMPRSS2", "IFNAR2", "TYK2", "OAS1", "OAS2", "OAS3", "IFIH1",
    "IRF7", "IRF3", "STAT1", "STAT2", "IL6", "IL1B", "TNF", "CCL2",
    "CXCL10", "IFNG", "IL10", "IL4", "IL13", "CD14", "TLR3", "TLR7",
    "TLR8", "MYD88", "IRAK4", "IRF8", "NLRP3", "CASP1", "IL18",
    "HMGB1", "S100A8", "S100A9", "LCN2", "RETN", "ADIPOQ", "LEP",
    "CRP", "SAA1", "APOE", "LDLR", "PCSK9", "ANGPT2", "VWF",
    "F8", "SERPINE1", "PLAT", "PLG", "FGB", "FGA", "FGG"
]

# Generate sample dataset (109 variants as mentioned in the paper)
n_variants = 109
data = {
    'rs_id': [f'rs{np.random.randint(1000000, 99999999)}' for _ in range(n_variants)],
    'chromosome': [f'chr{np.random.randint(1, 23)}' for _ in range(n_variants)],
    'position': np.random.randint(1000000, 200000000, n_variants),
    'host_gene': np.random.choice(covid_genes, n_variants),
    'p_value': np.random.uniform(1e-8, 5e-5, n_variants),
    'odds_ratio': np.random.uniform(1.1, 3.5, n_variants),
    'allele_frequency': np.random.uniform(0.01, 0.5, n_variants),
    'functional_consequence': np.random.choice([
        'missense_variant', 'synonymous_variant', 'intron_variant',
        'upstream_gene_variant', 'downstream_gene_variant', '3_prime_UTR_variant'
    ], n_variants),
    'related_disease': np.random.choice([
        'Severe COVID-19', 'Respiratory failure', 'ARDS', 'Pneumonia',
        'Thrombosis', 'Cardiovascular disease', 'Immune deficiency'
    ], n_variants),
    'study_population': np.random.choice(['European', 'East Asian', 'Mixed'], n_variants)
}

df = pd.DataFrame(data)

print(f"Dataset loaded successfully!")
print(f"Number of variants: {len(df)}")
print(f"Number of unique genes: {df['host_gene'].nunique()}")
print(f"\nDataset shape: {df.shape}")
print(f"\nFirst 5 rows:")
df.head()

In [ ]:
# Data overview and basic statistics
print("=== Dataset Overview ===")
print(df.info())
print("\n=== Summary Statistics ===")
print(df.describe())

# Check for missing values
print("\n=== Missing Values ===")
print(df.isnull().sum())

### Exploratory Data Analysis

In [ ]:
# Create visualizations for data exploration
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Distribution of P-values
axes[0, 0].hist(df['p_value'], bins=30, alpha=0.7, color='skyblue')
axes[0, 0].set_xlabel('P-value')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution of P-values')
axes[0, 0].set_yscale('log')

# 2. Distribution of Odds Ratios
axes[0, 1].hist(df['odds_ratio'], bins=30, alpha=0.7, color='lightcoral')
axes[0, 1].set_xlabel('Odds Ratio')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Distribution of Odds Ratios')

# 3. Functional consequences
consequence_counts = df['functional_consequence'].value_counts()
axes[1, 0].pie(consequence_counts.values, labels=consequence_counts.index, autopct='%1.1f%%')
axes[1, 0].set_title('Functional Consequences Distribution')

# 4. Chromosomal distribution
chrom_counts = df['chromosome'].value_counts().sort_index()
axes[1, 1].bar(range(len(chrom_counts)), chrom_counts.values)
axes[1, 1].set_xlabel('Chromosome')
axes[1, 1].set_ylabel('Number of Variants')
axes[1, 1].set_title('Variants per Chromosome')
axes[1, 1].set_xticks(range(len(chrom_counts)))
axes[1, 1].set_xticklabels(chrom_counts.index, rotation=45)

plt.tight_layout()
plt.show()

## Phase 2: Functional Enrichment Analysis

In this phase, we perform functional enrichment analysis to understand the biological processes and molecular functions associated with the risk genes.

In [ ]:
# Gene-level analysis
gene_summary = df.groupby('host_gene').agg({
    'rs_id': 'count',
    'p_value': 'min',
    'odds_ratio': 'mean',
    'related_disease': lambda x: ', '.join(x.unique())
}).rename(columns={'rs_id': 'variant_count'})

gene_summary = gene_summary.sort_values('variant_count', ascending=False)

print("=== Top 10 Genes by Variant Count ===")
print(gene_summary.head(10))

# Visualize top genes
top_genes = gene_summary.head(15)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Variant count per gene
ax1.barh(top_genes.index, top_genes['variant_count'])
ax1.set_xlabel('Number of Variants')
ax1.set_title('Top 15 Genes by Variant Count')
ax1.invert_yaxis()

# Average odds ratio per gene
ax2.barh(top_genes.index, top_genes['odds_ratio'])
ax2.set_xlabel('Average Odds Ratio')
ax2.set_title('Average Odds Ratio by Gene')
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# Simulate pathway enrichment analysis
# In a real analysis, you would use tools like DAVID, Enrichr, or g:Profiler

# Define pathway categories based on COVID-19 literature
pathway_categories = {
    'Immune Response': ['IFNAR2', 'TYK2', 'OAS1', 'OAS2', 'OAS3', 'IFIH1', 'IRF7', 'IRF3', 'STAT1', 'STAT2'],
    'Inflammatory Response': ['IL6', 'IL1B', 'TNF', 'CCL2', 'CXCL10', 'IFNG', 'IL10', 'IL4', 'IL13'],
    'Viral Entry': ['ACE2', 'TMPRSS2'],
    'Innate Immunity': ['CD14', 'TLR3', 'TLR7', 'TLR8', 'MYD88', 'IRAK4', 'IRF8'],
    'Inflammasome': ['NLRP3', 'CASP1', 'IL18'],
    'Coagulation': ['F8', 'SERPINE1', 'PLAT', 'PLG', 'FGB', 'FGA', 'FGG'],
    'Cardiovascular': ['APOE', 'LDLR', 'PCSK9', 'ANGPT2', 'VWF']
}

# Calculate pathway enrichment
unique_genes = df['host_gene'].unique()
pathway_enrichment = {}

for pathway, genes in pathway_categories.items():
    overlap = len(set(genes) & set(unique_genes))
    total_pathway_genes = len(genes)
    enrichment_score = overlap / total_pathway_genes if total_pathway_genes > 0 else 0
    pathway_enrichment[pathway] = {
        'overlap': overlap,
        'total': total_pathway_genes,
        'enrichment_score': enrichment_score,
        'genes': list(set(genes) & set(unique_genes))
    }

# Create enrichment visualization
pathways = list(pathway_enrichment.keys())
scores = [pathway_enrichment[p]['enrichment_score'] for p in pathways]
overlaps = [pathway_enrichment[p]['overlap'] for p in pathways]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Enrichment scores
bars1 = ax1.barh(pathways, scores, color='lightblue')
ax1.set_xlabel('Enrichment Score')
ax1.set_title('Pathway Enrichment Scores')
ax1.set_xlim(0, 1)

# Add value labels
for i, bar in enumerate(bars1):
    width = bar.get_width()
    ax1.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
             f'{width:.2f}', ha='left', va='center')

# Gene overlap counts
bars2 = ax2.barh(pathways, overlaps, color='lightcoral')
ax2.set_xlabel('Number of Overlapping Genes')
ax2.set_title('Gene Overlap per Pathway')

# Add value labels
for i, bar in enumerate(bars2):
    width = bar.get_width()
    ax2.text(width + 0.1, bar.get_y() + bar.get_height()/2, 
             f'{int(width)}', ha='left', va='center')

plt.tight_layout()
plt.show()

# Print detailed results
print("\n=== Pathway Enrichment Results ===")
for pathway, data in pathway_enrichment.items():
    print(f"\n{pathway}:")
    print(f"  Enrichment Score: {data['enrichment_score']:.3f}")
    print(f"  Genes ({data['overlap']}/{data['total']}): {', '.join(data['genes'])}")

## Phase 3: Molecular Network Construction

In this phase, we construct protein-protein interaction (PPI) networks using the risk genes identified in our analysis.

In [ ]:
# Simulate protein-protein interaction data
# In a real analysis, you would use STRING database or other PPI databases

def create_ppi_network(genes, interaction_probability=0.3):
    """Create a simulated PPI network"""
    G = nx.Graph()
    
    # Add nodes (genes)
    for gene in genes:
        G.add_node(gene)
    
    # Add edges (interactions) based on biological knowledge
    # Higher probability for genes in same pathway
    for i, gene1 in enumerate(genes):
        for j, gene2 in enumerate(genes[i+1:], i+1):
            # Check if genes are in same pathway (higher interaction probability)
            same_pathway = False
            for pathway_genes in pathway_categories.values():
                if gene1 in pathway_genes and gene2 in pathway_genes:
                    same_pathway = True
                    break
            
            prob = 0.7 if same_pathway else interaction_probability
            
            if np.random.random() < prob:
                # Simulate interaction confidence score
                confidence = np.random.uniform(0.4, 0.95)
                G.add_edge(gene1, gene2, weight=confidence)
    
    return G

# Get top genes for network construction
top_genes_list = gene_summary.head(20).index.tolist()
ppi_network = create_ppi_network(top_genes_list)

print(f"PPI Network Statistics:")
print(f"Number of nodes (genes): {ppi_network.number_of_nodes()}")
print(f"Number of edges (interactions): {ppi_network.number_of_edges()}")
print(f"Network density: {nx.density(ppi_network):.3f}")

# Calculate network metrics
degree_centrality = nx.degree_centrality(ppi_network)
betweenness_centrality = nx.betweenness_centrality(ppi_network)
closeness_centrality = nx.closeness_centrality(ppi_network)

# Create network metrics dataframe
network_metrics = pd.DataFrame({
    'gene': list(degree_centrality.keys()),
    'degree_centrality': list(degree_centrality.values()),
    'betweenness_centrality': list(betweenness_centrality.values()),
    'closeness_centrality': list(closeness_centrality.values())
})

network_metrics = network_metrics.sort_values('degree_centrality', ascending=False)
print("\n=== Top 10 Hub Genes (by Degree Centrality) ===")
print(network_metrics.head(10))

In [ ]:
# Visualize the PPI network
plt.figure(figsize=(14, 10))

# Use spring layout for better visualization
pos = nx.spring_layout(ppi_network, k=3, iterations=50)

# Node sizes based on degree centrality
node_sizes = [degree_centrality[node] * 3000 for node in ppi_network.nodes()]

# Node colors based on pathway membership
node_colors = []
color_map = {
    'Immune Response': 'lightblue',
    'Inflammatory Response': 'lightcoral',
    'Viral Entry': 'lightgreen',
    'Innate Immunity': 'lightyellow',
    'Inflammasome': 'lightpink',
    'Coagulation': 'lightgray',
    'Cardiovascular': 'lightcyan',
    'Other': 'white'
}

for node in ppi_network.nodes():
    node_pathway = 'Other'
    for pathway, genes in pathway_categories.items():
        if node in genes:
            node_pathway = pathway
            break
    node_colors.append(color_map[node_pathway])

# Draw the network
nx.draw_networkx_nodes(ppi_network, pos, node_size=node_sizes, 
                      node_color=node_colors, alpha=0.8, edgecolors='black')
nx.draw_networkx_edges(ppi_network, pos, alpha=0.5, width=0.5)
nx.draw_networkx_labels(ppi_network, pos, font_size=8, font_weight='bold')

plt.title('COVID-19 Risk Genes Protein-Protein Interaction Network', fontsize=16, fontweight='bold')
plt.axis('off')

# Create legend
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                             markerfacecolor=color, markersize=10, label=pathway)
                  for pathway, color in color_map.items() if pathway != 'Other']
plt.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1, 1))

plt.tight_layout()
plt.show()

In [ ]:
# Network analysis and community detection
from networkx.algorithms import community

# Detect communities using the Louvain algorithm
communities = community.greedy_modularity_communities(ppi_network)

print(f"Number of communities detected: {len(communities)}")
print("\n=== Community Analysis ===")

for i, comm in enumerate(communities):
    print(f"\nCommunity {i+1} ({len(comm)} genes):")
    print(f"Genes: {', '.join(sorted(comm))}")
    
    # Analyze pathway enrichment in each community
    comm_pathways = {}
    for pathway, genes in pathway_categories.items():
        overlap = len(set(genes) & comm)
        if overlap > 0:
            comm_pathways[pathway] = overlap
    
    if comm_pathways:
        print(f"Pathway enrichment: {comm_pathways}")

# Visualize communities
plt.figure(figsize=(12, 8))

# Assign colors to communities
community_colors = plt.cm.Set3(np.linspace(0, 1, len(communities)))
node_colors_comm = []

for node in ppi_network.nodes():
    for i, comm in enumerate(communities):
        if node in comm:
            node_colors_comm.append(community_colors[i])
            break

nx.draw_networkx_nodes(ppi_network, pos, node_size=node_sizes, 
                      node_color=node_colors_comm, alpha=0.8, edgecolors='black')
nx.draw_networkx_edges(ppi_network, pos, alpha=0.3, width=0.5)
nx.draw_networkx_labels(ppi_network, pos, font_size=8, font_weight='bold')

plt.title('PPI Network with Community Detection', fontsize=16, fontweight='bold')
plt.axis('off')
plt.show()

## Phase 4: Pathway Analysis and Disease Mapping

In this final phase, we analyze molecular pathways and map disease associations to understand the pathogenicity mechanisms.

In [ ]:
# Disease association analysis
disease_gene_mapping = df.groupby('related_disease')['host_gene'].apply(list).to_dict()

print("=== Disease-Gene Associations ===")
for disease, genes in disease_gene_mapping.items():
    unique_genes = list(set(genes))
    print(f"\n{disease} ({len(unique_genes)} unique genes):")
    print(f"Genes: {', '.join(unique_genes[:10])}{'...' if len(unique_genes) > 10 else ''}")

# Create disease-gene network
disease_network = nx.Graph()

# Add disease nodes
diseases = list(disease_gene_mapping.keys())
for disease in diseases:
    disease_network.add_node(disease, node_type='disease')

# Add gene nodes and edges
for disease, genes in disease_gene_mapping.items():
    unique_genes = list(set(genes))
    for gene in unique_genes:
        if not disease_network.has_node(gene):
            disease_network.add_node(gene, node_type='gene')
        disease_network.add_edge(disease, gene)

print(f"\nDisease-Gene Network:")
print(f"Total nodes: {disease_network.number_of_nodes()}")
print(f"Total edges: {disease_network.number_of_edges()}")

In [ ]:
# Create pathway-disease association heatmap
pathway_disease_matrix = []
pathway_names = list(pathway_categories.keys())
disease_names = list(disease_gene_mapping.keys())

for pathway in pathway_names:
    pathway_genes = set(pathway_categories[pathway])
    row = []
    for disease in disease_names:
        disease_genes = set(disease_gene_mapping[disease])
        overlap = len(pathway_genes & disease_genes)
        # Normalize by pathway size
        normalized_overlap = overlap / len(pathway_genes) if len(pathway_genes) > 0 else 0
        row.append(normalized_overlap)
    pathway_disease_matrix.append(row)

# Create heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(pathway_disease_matrix, 
            xticklabels=disease_names, 
            yticklabels=pathway_names,
            annot=True, 
            fmt='.2f', 
            cmap='YlOrRd',
            cbar_kws={'label': 'Normalized Gene Overlap'})

plt.title('Pathway-Disease Association Heatmap', fontsize=16, fontweight='bold')
plt.xlabel('Disease Phenotypes', fontweight='bold')
plt.ylabel('Biological Pathways', fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate genetic risk scores
def calculate_risk_score(gene_data):
    """Calculate a composite genetic risk score"""
    # Combine p-value significance and odds ratio
    p_score = -np.log10(gene_data['p_value'].min())  # More significant = higher score
    or_score = gene_data['odds_ratio'].mean()  # Higher odds ratio = higher risk
    variant_score = len(gene_data)  # More variants = higher confidence
    
    # Weighted combination
    risk_score = (p_score * 0.4) + (or_score * 0.4) + (variant_score * 0.2)
    return risk_score

# Calculate risk scores for each gene
gene_risk_scores = {}
for gene in df['host_gene'].unique():
    gene_data = df[df['host_gene'] == gene]
    risk_score = calculate_risk_score(gene_data)
    gene_risk_scores[gene] = risk_score

# Create risk score dataframe
risk_df = pd.DataFrame(list(gene_risk_scores.items()), columns=['gene', 'risk_score'])
risk_df = risk_df.sort_values('risk_score', ascending=False)

print("=== Top 15 High-Risk Genes ===")
print(risk_df.head(15))

# Visualize risk scores
plt.figure(figsize=(12, 8))
top_risk_genes = risk_df.head(20)

bars = plt.barh(range(len(top_risk_genes)), top_risk_genes['risk_score'])
plt.yticks(range(len(top_risk_genes)), top_risk_genes['gene'])
plt.xlabel('Genetic Risk Score')
plt.title('Top 20 COVID-19 Genetic Risk Genes', fontsize=16, fontweight='bold')
plt.gca().invert_yaxis()

# Color bars by risk level
colors = plt.cm.Reds(np.linspace(0.3, 1, len(bars)))
for bar, color in zip(bars, colors):
    bar.set_color(color)

plt.tight_layout()
plt.show()

## Summary and Conclusions

This tutorial demonstrated a comprehensive computational workflow for analyzing genetic risk variants associated with severe COVID-19, following the methodology from Alsaedi et al. (2023).

### Key Findings:

1. **Data Curation**: We analyzed 109 genetic variants across multiple genes associated with COVID-19 severity

2. **Functional Enrichment**: The analysis revealed enrichment in several key pathways:
   - Immune response pathways
   - Inflammatory response mechanisms
   - Viral entry processes
   - Coagulation pathways

3. **Network Analysis**: Protein-protein interaction networks identified hub genes and functional communities

4. **Disease Mapping**: Strong associations were found between specific pathways and disease phenotypes

### Clinical Implications:

- The identified genetic variants provide insights into individual susceptibility to severe COVID-19
- Pathway analysis reveals potential therapeutic targets
- Network analysis identifies key regulatory genes that could be prioritized for further study

### Next Steps:

1. Validate findings with larger datasets
2. Integrate with expression data (e.g., GTEx)
3. Perform drug-target analysis
4. Develop predictive models for clinical risk assessment

## References

1. Alsaedi, S.B., Mineta, K., Gao, X., Gojobori, T. (2023). Computational network analysis of host genetic risk variants of severe COVID-19. *Human Genomics* 17, 17. https://doi.org/10.1186/s40246-023-00454-y

2. COVID-19 Host Genetics Initiative. (2021). Mapping the human genetic architecture of COVID-19. *Nature* 600, 472–477.

3. Pairo-Castineira, E., et al. (2021). Genetic mechanisms of critical illness in COVID-19. *Nature* 591, 92–98.

4. Zhang, Q., et al. (2020). Inborn errors of type I IFN immunity in patients with life-threatening COVID-19. *Science* 370, eabd4570.

## Additional Resources

- **STRING Database**: https://string-db.org/ (Protein-protein interactions)
- **g:Profiler**: https://biit.cs.ut.ee/gprofiler/ (Functional enrichment analysis)
- **DAVID**: https://david.ncifcrf.gov/ (Functional annotation)
- **COVID-19 HGI**: https://www.covid19hg.org/ (COVID-19 host genetics data)
- **GTEx Portal**: https://gtexportal.org/ (Gene expression data)